# CNN 84.43% , 0.82337

In [1]:
import keras
import matplotlib.image as mpimg
import numpy
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import tensorflow as tf

Using TensorFlow backend.
/Users/besmaelketroussi/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - numpy.min(img)
    rimg = (rimg / numpy.max(rimg) * PIXEL_DEPTH).round().astype(numpy.uint8)
    return rimg

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    gt_patches = [img_crop(gt_imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = numpy.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
    labels = numpy.asarray([value_to_class(numpy.mean(data[i])) for i in range(len(data))])

    # Convert to dense 1-hot representation.
    return labels.astype(numpy.float32)

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch
    df = numpy.sum(v)
    if df > foreground_threshold:
        return [0, 1]
    else:
        return [1, 0]
        
def extract_data(filename, num_images):
    """Extract the images into a 4D tensor [image index, y, x, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    N_PATCHES_PER_IMAGE = (IMG_WIDTH/IMG_PATCH_SIZE)*(IMG_HEIGHT/IMG_PATCH_SIZE)

    img_patches = [img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    return numpy.asarray(data)


In [3]:
from sklearn.model_selection import train_test_split


NUM_CHANNELS = 3 # RGB images
PIXEL_DEPTH = 255
NUM_LABELS = 2
TRAINING_SIZE = 100
VALIDATION_SIZE = 5  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 128 # 64
NUM_EPOCHS = 150
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 1000

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 16

root_dir = "training/"
image_dir = root_dir + "images/"
gt_dir = root_dir + "groundtruth/"

train_data = extract_data(image_dir, TRAINING_SIZE)
train_labels = extract_labels(gt_dir, TRAINING_SIZE)


X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)


Loading training/images/satImage_001.png
Loading training/images/satImage_002.png
Loading training/images/satImage_003.png
Loading training/images/satImage_004.png
Loading training/images/satImage_005.png
Loading training/images/satImage_006.png
Loading training/images/satImage_007.png
Loading training/images/satImage_008.png
Loading training/images/satImage_009.png
Loading training/images/satImage_010.png
Loading training/images/satImage_011.png
Loading training/images/satImage_012.png
Loading training/images/satImage_013.png
Loading training/images/satImage_014.png
Loading training/images/satImage_015.png
Loading training/images/satImage_016.png
Loading training/images/satImage_017.png
Loading training/images/satImage_018.png
Loading training/images/satImage_019.png
Loading training/images/satImage_020.png
Loading training/images/satImage_021.png
Loading training/images/satImage_022.png
Loading training/images/satImage_023.png
Loading training/images/satImage_024.png
Loading training

In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU, PReLU

 


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(16, 16,3), padding='same', activation='relu', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(128, (3, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
#model.add(Dense(32, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))
#model.add(Activation('sigmoid'))
#model.add(Dropout(0.5))
#model.add(Dense(num_classes, activation='softmax'))



epochs = NUM_EPOCHS
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=1e-6, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 16, 16, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
__________

In [5]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=16)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 50000 samples, validate on 12500 samples
Epoch 1/150
50000/50000 [==============================] - 45s 893us/step - loss: 0.5288 - acc: 0.7414 - val_loss: 0.5048 - val_acc: 0.7373
Epoch 2/150
50000/50000 [==============================] - 39s 783us/step - loss: 0.4946 - acc: 0.7518 - val_loss: 0.4991 - val_acc: 0.7586
Epoch 3/150
50000/50000 [==============================] - 39s 782us/step - loss: 0.4623 - acc: 0.7728 - val_loss: 0.4751 - val_acc: 0.7574
Epoch 4/150
50000/50000 [==============================] - 39s 783us/step - loss: 0.4296 - acc: 0.7904 - val_loss: 0.3978 - val_acc: 0.8114
Epoch 5/150
50000/50000 [==============================] - 39s 783us/step - loss: 0.4170 - acc: 0.7982 - val_loss: 0.4216 - val_acc: 0.7961
Epoch 6/150
50000/50000 [==============================] - 39s 789us/step - loss: 0.4018 - acc: 0.8064 - val_loss: 0.3925 - val_acc: 0.8121
Epoch 7/150
50000/50000 [==============================] - 39s 785us/step - loss: 0.3916 - acc: 0.8154 - val_l

Epoch 59/150
50000/50000 [==============================] - 39s 775us/step - loss: 0.2258 - acc: 0.9001 - val_loss: 0.3746 - val_acc: 0.8512
Epoch 60/150
50000/50000 [==============================] - 39s 773us/step - loss: 0.2265 - acc: 0.8995 - val_loss: 0.3875 - val_acc: 0.8484
Epoch 61/150
50000/50000 [==============================] - 39s 774us/step - loss: 0.2203 - acc: 0.9032 - val_loss: 0.3970 - val_acc: 0.8525
Epoch 62/150
50000/50000 [==============================] - 39s 774us/step - loss: 0.2218 - acc: 0.9019 - val_loss: 0.3839 - val_acc: 0.8416
Epoch 63/150
50000/50000 [==============================] - 39s 774us/step - loss: 0.2198 - acc: 0.9032 - val_loss: 0.3958 - val_acc: 0.8462
Epoch 64/150
50000/50000 [==============================] - 39s 772us/step - loss: 0.2175 - acc: 0.9039 - val_loss: 0.4083 - val_acc: 0.8465
Epoch 65/150
50000/50000 [==============================] - 39s 774us/step - loss: 0.2155 - acc: 0.9054 - val_loss: 0.3771 - val_acc: 0.8470
Epoch 66/150


Epoch 117/150
50000/50000 [==============================] - 39s 774us/step - loss: 0.1583 - acc: 0.9355 - val_loss: 0.4919 - val_acc: 0.8498
Epoch 118/150
50000/50000 [==============================] - 39s 771us/step - loss: 0.1593 - acc: 0.9352 - val_loss: 0.4739 - val_acc: 0.8479
Epoch 119/150
50000/50000 [==============================] - 38s 769us/step - loss: 0.1564 - acc: 0.9362 - val_loss: 0.4772 - val_acc: 0.8481
Epoch 120/150
50000/50000 [==============================] - 40s 791us/step - loss: 0.1548 - acc: 0.9359 - val_loss: 0.5034 - val_acc: 0.8481
Epoch 121/150
50000/50000 [==============================] - 39s 777us/step - loss: 0.1560 - acc: 0.9363 - val_loss: 0.5012 - val_acc: 0.8490
Epoch 122/150
50000/50000 [==============================] - 39s 771us/step - loss: 0.1535 - acc: 0.9370 - val_loss: 0.4515 - val_acc: 0.8441
Epoch 123/150
50000/50000 [==============================] - 39s 770us/step - loss: 0.1523 - acc: 0.9380 - val_loss: 0.4802 - val_acc: 0.8493
Epoch 

# Prediction

In [11]:
import numpy as np
def label_to_img(imgwidth, imgheight, w, h, labels):
    array_labels = numpy.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if labels[idx][0] > 0.5:
                l = 0
            else:
                l = 1
            array_labels[j:j+w, i:i+h] = l
            idx = idx + 1
    return array_labels

def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = numpy.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = numpy.zeros((w, h, 3), dtype=numpy.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = numpy.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def bw(gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = gt_img #np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)          
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        cimg = gt_img_3c
    return cimg

    # Get prediction for given input image 
def get_prediction(img):
    data = numpy.asarray(img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    output_prediction = model.predict(data)
    img_prediction = label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

    # Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(filename, image_idx):
    
    imageid = "test_" + str(image_idx)
    image_filename = filename + imageid + ".png"
    img = mpimg.imread(image_filename)
    
    img_prediction = get_prediction(img)
    #cimg = concatenate_images(img, img_prediction)
    #cimg = Image.fromarray(bw(img_prediction))

    #return cimg
    return img_prediction

submission_filename = 'submission.csv'
image_filenames = []

for i in range(1, 51):
    'test_set_images/test_'
    test_data_filename = 'test_set_images/test_' + str(i) + '/'
    pimg = get_prediction_with_groundtruth(test_data_filename, i)
    image_filename = 'test_set_images/pred_' + str(i) + '.png'
    Image.fromarray(bw(pimg)).save(image_filename)
    print(image_filename)
    image_filenames.append(image_filename)


test_set_images/pred_1.png
test_set_images/pred_2.png
test_set_images/pred_3.png
test_set_images/pred_4.png
test_set_images/pred_5.png
test_set_images/pred_6.png
test_set_images/pred_7.png
test_set_images/pred_8.png
test_set_images/pred_9.png
test_set_images/pred_10.png
test_set_images/pred_11.png
test_set_images/pred_12.png
test_set_images/pred_13.png
test_set_images/pred_14.png
test_set_images/pred_15.png
test_set_images/pred_16.png
test_set_images/pred_17.png
test_set_images/pred_18.png
test_set_images/pred_19.png
test_set_images/pred_20.png
test_set_images/pred_21.png
test_set_images/pred_22.png
test_set_images/pred_23.png
test_set_images/pred_24.png
test_set_images/pred_25.png
test_set_images/pred_26.png
test_set_images/pred_27.png
test_set_images/pred_28.png
test_set_images/pred_29.png
test_set_images/pred_30.png
test_set_images/pred_31.png
test_set_images/pred_32.png
test_set_images/pred_33.png
test_set_images/pred_34.png
test_set_images/pred_35.png
test_set_images/pred_36.png
t

In [12]:
from mask_to_submission import *
masks_to_submission(submission_filename, *image_filenames)